### Chargement Packages

In [ ]:
# Chargement package
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

### Fonctions

In [15]:
def recodage(dataset):
   # Liste des colonnes à transtyper en numeric
  var_tofloat = ['int_corr','income','pf_o_att',	'pf_o_sin',	'pf_o_int',	'pf_o_fun',	'pf_o_amb',	'pf_o_sha', 'attr1_1', 'sinc1_1',	'intel1_1',	'fun1_1',	'amb1_1',	'shar1_1']

  for i in var_tofloat:
    dataset[i] = dataset[i].str.replace(",","")
    dataset[i] = pd.to_numeric(dataset[i], downcast="float")
  # Liste des colonnes à transtyper en object
  var_toobject = ['gender','idg', 'condtn',	'wave', 'position',	'positin1','match','samerace','race_o','dec_o','field_cd', 'race', 'goal','date', 'go_out', 'career_c']

  for i in var_toobject:
    dataset[i] = dataset[i].astype("object")

  # Remove field career , codage de field et career
  df_res = dataset.drop(columns=['field', 'career','dec_o', 'order', 'positin1', 'position', 'round', 'wave'])

  return df_res

In [23]:
def recodage_withdec_o(dataset):
   # Liste des colonnes à transtyper en numeric
  var_tofloat = ['int_corr','income','pf_o_att',	'pf_o_sin',	'pf_o_int',	'pf_o_fun',	'pf_o_amb',	'pf_o_sha', 'attr1_1', 'sinc1_1',	'intel1_1',	'fun1_1',	'amb1_1',	'shar1_1']

  for i in var_tofloat:
    dataset[i] = dataset[i].str.replace(",","")
    dataset[i] = pd.to_numeric(dataset[i], downcast="float")
  # Liste des colonnes à transtyper en object
  var_toobject = ['gender','idg', 'condtn',	'wave', 'position',	'positin1','match','samerace','race_o','dec_o','field_cd', 'race', 'goal','date', 'go_out', 'career_c']

  for i in var_toobject:
    dataset[i] = dataset[i].astype("object")

  # Remove field career , codage de field et career
  df_res = dataset.drop(columns=['field', 'career', 'order', 'positin1', 'position', 'round'])

  return df_res

In [ ]:
def remove_na(dataset):
  # Quanti
  df_quanti = dataset.select_dtypes(exclude='object').copy()
  imputer = KNNImputer(n_neighbors=2, weights="distance")
  df_quanti = pd.DataFrame(imputer.fit_transform(df_quanti), columns = df_quanti.columns)

  #print(df_quanti.info())
  # Quali 
  df_quali = dataset.select_dtypes(include='object').copy()
  #print(df_quali.info())
  df_quali.iloc[:,5:].fillna("Missing value", inplace = True)

  #print(df_quali.info())
  # Combinaison des deux datasets 
  df_clean = pd.concat([df_quanti, df_quali.reindex(df_quanti.index)], axis=1)

  print("Format : " )
  print(df_clean.shape)
  print("Nombre de NaN : ")
  print(df_clean.isna().sum().sum())
  return df_clean

In [ ]:
def clean_data(dataset):
  data_reco = recodage(dataset)


  data_res = remove_na(data_reco)
  return data_res

In [22]:
def clean_data_withdec_o(dataset):
  data_reco = recodage_withdec_o(dataset)


  data_res = remove_na(data_reco)
  return data_res

### Chargement Dataset

In [8]:
# Connexion à ggdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Import du fichier via ggdrive
file_path = "/content/drive/MyDrive/M2_SISE/Python_M2/train.csv" # Lien perso ggdrive
df_train = pd.read_csv(file_path, sep = ';')

### Application des formules 

In [10]:
df = df_train.copy()
df_nett = clean_data(df)

Format : 
(6804, 62)
Nombre de NaN : 
0


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [24]:
df = df_train.copy()
df_nett_deco = clean_data_withdec_o(df)

Format : 
(6804, 64)
Nombre de NaN : 
0


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [13]:
df_nett.shape

(6804, 62)

### Export

In [ ]:
from google.colab import files
df_nett.to_csv('df_clean.csv') 
files.download('df_clean.csv')

In [25]:
from google.colab import files
df_nett_deco.to_csv('df_clean_deco.csv') 
files.download('df_clean_deco.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>